In [1]:
import numpy as np
import dicom
import sys
sys.path.append('..')
import os
from AlgoEngine.utils import getContours
from AlgoEngine.ovh import getOVHDistances
from math import sqrt

## Inputs to Function

We compute OVH distances for PTV and bladder in this test

In [2]:
BASE_DIR = '/home/radiation/RadiationTherapyDecisionSupport/data/'
StudyID = 'UCLA_PR_5'

ctFilenames = [fl for fl in os.listdir(BASE_DIR + StudyID) if 'CT.' in fl]
numImages = len(ctFilenames)

sampleCTImage = dicom.read_file(BASE_DIR + StudyID + '/' + ctFilenames[0])
width = sampleCTImage.Columns
height = sampleCTImage.Rows
row_spacing = float(sampleCTImage.PixelSpacing[0])
column_spacing = float(sampleCTImage.PixelSpacing[1])
slice_spacing = float(sampleCTImage.SliceThickness)

block_shape = (width, height, numImages)
slice_position_z = np.zeros((numImages)).astype(np.float32) 

for i, fl in enumerate(ctFilenames):
    slice_position_z[i] = dicom.read_file(BASE_DIR + StudyID + '/' + fl).ImagePositionPatient[2]
slice_position_z = np.sort(slice_position_z)[::-1]

In [3]:
structureset = dicom.read_file(BASE_DIR + StudyID + '/structureset.dcm')
PTV_ROI_NUM = -1
OAR_ROI_NUM = -1

for n in range(0, len(structureset.StructureSetROISequence)):
    if structureset.StructureSetROISequence[n].ROIName == 'PTV':
        PTV_ROI_NUM = structureset.StructureSetROISequence[n].ROINumber
    elif structureset.StructureSetROISequence[n].ROIName == 'Bladder':
        OAR_ROI_NUM = structureset.StructureSetROISequence[n].ROINumber

In [4]:
roiNumPlanes = len(structureset.ROIContourSequence[PTV_ROI_NUM].ContourSequence) 

contour_data = {} 
image_orientation = {} 
image_position = {} 
pixel_spacing = {} 

for index in range(0, roiNumPlanes):
    
    imageSOP = structureset.ROIContourSequence[PTV_ROI_NUM].ContourSequence[index].ContourImageSequence[0].ReferencedSOPInstanceUID
    
    planeContourData = np.array(structureset.ROIContourSequence[PTV_ROI_NUM].ContourSequence[index].ContourData)
    planeContourData = planeContourData.reshape(planeContourData.shape[0] // 3 , 3)
    
    contour_data[imageSOP] = planeContourData
    imagei = dicom.read_file(BASE_DIR + StudyID + '/CT.' + imageSOP + '.dcm')
    
    image_orientation[imageSOP] = imagei.ImageOrientationPatient
    image_position[imageSOP] = imagei.ImagePositionPatient
    pixel_spacing[imageSOP] = imagei.PixelSpacing
ptv_contour_block, ptv_roi_block = getContours(block_shape, slice_position_z, contour_data, image_orientation, image_position, pixel_spacing)

In [5]:
roiNumPlanes = len(structureset.ROIContourSequence[OAR_ROI_NUM].ContourSequence) 

contour_data = {}
image_orientation = {}
image_position = {} 
pixel_spacing = {}

for index in range(0, roiNumPlanes):
    
    imageSOP = structureset.ROIContourSequence[OAR_ROI_NUM].ContourSequence[index].ContourImageSequence[0].ReferencedSOPInstanceUID
    planeContourData = np.array(structureset.ROIContourSequence[OAR_ROI_NUM].ContourSequence[index].ContourData)
    planeContourData = planeContourData.reshape(planeContourData.shape[0] // 3 , 3)
    
    contour_data[imageSOP] = planeContourData
    imagei = dicom.read_file(BASE_DIR + StudyID + '/CT.' + imageSOP + '.dcm')
    
    image_orientation[imageSOP] = imagei.ImageOrientationPatient
    image_position[imageSOP] = imagei.ImagePositionPatient
    pixel_spacing[imageSOP] = imagei.PixelSpacing
_, oar_roi_block = getContours(block_shape, slice_position_z, contour_data, image_orientation, image_position, pixel_spacing)

## Function Starts Here

In [6]:
oar_dists = getOVHDistances(oar_roi_block, ptv_contour_block, ptv_roi_block, row_spacing, column_spacing, slice_spacing)

## Testing Output

Note that because we lack access to an actual validation example, we test using the assertion that for 5 random places in the OAR, there is not a place in the PTV that is closer than the marked distance. 

In [7]:
oar_row, oar_col, oar_slice = np.nonzero(oar_dists)
ptv_row, ptv_col, ptv_slice = np.nonzero(ptv_contour_block)

alpha = column_spacing / row_spacing
beta =  slice_spacing / row_spacing

test_passed = True

indices = np.arange(oar_row.shape[0])
np.random.shuffle(indices)

for n in range(0, 30):
    oar_point = (oar_row[indices[n]], oar_col[indices[n]], oar_slice[indices[n]])
    for i in range(0, ptv_row.shape[0]):
        dist = sqrt( (oar_point[0] - ptv_row[i])**2  + 
                      alpha*(oar_point[1] - ptv_col[i])**2 + 
                      beta*(oar_point[2] - ptv_slice[i])**2)
        
        # allclose handles errors from comparing single precision to double precision number
        if dist < oar_dists[oar_point] and not np.allclose(np.array(dist), np.array(oar_dists[oar_point])):
            test_passed = False

print(test_passed)

True
